In [1]:
!git clone https://github.com/kpu/kenlm.git

Cloning into 'kenlm'...
remote: Enumerating objects: 14102, done.
remote: Counting objects: 100% (415/415), done.
remote: Compressing objects: 100% (291/291), done.
remote: Total 14102 (delta 126), reused 370 (delta 110), pack-reused 13687
Receiving objects: 100% (14102/14102), 5.89 MiB | 19.92 MiB/s, done.
Resolving deltas: 100% (8006/8006), done.


In [2]:
%cd /content/kenlm
!mkdir -p build
%cd build
!cmake ..

/content/kenlm
/content/kenlm/build
-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Could NOT find Eigen3 (missing: Eigen3_DIR)
-- Looking for pthread.h
-- Looking for pthread.h - found
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD - Failed
-- Looking for pthread_create in pthreads
-- Looking for pthread_create in pthreads - not found
-- Looking for pthread_create in pthread
-- Looking for pthread_create in pthread - found
-- Found Threads: TRUE  
-- Found Boost: /usr/include (found suitable versi

In [3]:
!make -j 8

[  1%] Building CXX object util/CMakeFiles/kenlm_util.dir/double-conversion/strtod.cc.o
[  2%] Building CXX object util/CMakeFiles/kenlm_util.dir/double-conversion/fast-dtoa.cc.o
[  3%] Building CXX object util/CMakeFiles/kenlm_util.dir/double-conversion/bignum-dtoa.cc.o
[  5%] Building CXX object util/CMakeFiles/kenlm_util.dir/double-conversion/cached-powers.cc.o
[  6%] Building CXX object util/CMakeFiles/kenlm_util.dir/double-conversion/double-to-string.cc.o
[  7%] Building CXX object util/CMakeFiles/kenlm_util.dir/double-conversion/fixed-dtoa.cc.o
[  8%] Building CXX object util/CMakeFiles/kenlm_util.dir/double-conversion/string-to-double.cc.o
[ 10%] Building CXX object util/CMakeFiles/kenlm_util.dir/double-conversion/bignum.cc.o
[ 11%] Building CXX object util/CMakeFiles/kenlm_util.dir/stream/chain.cc.o
[ 12%] Building CXX object util/CMakeFiles/kenlm_util.dir/stream/count_records.cc.o
[ 13%] Building CXX object util/CMakeFiles/kenlm_util.dir/stream/io.cc.o
[ 15%] Building CXX obje

In [4]:
# Create file
!echo -e "hello world everyone\ngood afternoon ladies and gentleman\nmy friend in college\nhave a nice day\nspy and family at the same time\nwe do not do that here " >> text.txt

In [5]:
# you don't need --discount_fallback if you use your real data (which may huge)
!bin/lmplz -o 2 -S 80% -T /tmp <text.txt >text.arpa --discount_fallback

=== 1/5 Counting and sorting n-grams ===
Reading /content/kenlm/build/text.txt
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
tcmalloc: large alloc 2964283392 bytes == 0x55fab9e32000 @  0x7f5c51ea21e7 0x55fab782c912 0x55fab77c762e 0x55fab77a641b 0x55fab7792176 0x7f5c5003bc87 0x55fab7793cda
tcmalloc: large alloc 7904755712 bytes == 0x55fb6a928000 @  0x7f5c51ea21e7 0x55fab782c912 0x55fab781b93a 0x55fab781c378 0x55fab77a6438 0x55fab7792176 0x7f5c5003bc87 0x55fab7793cda
****************************************************************************************************
Unigram tokens 29 types 30
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:360 2:10894196736
tcmalloc: large alloc 10894196736 bytes == 0x55fd42460000 @  0x7f5c51ea21e7 0x55fab782c912 0x55fab781b93a 0x55fab781c378 0x55fab77a6a07 0x55fab7792176 0x7f5c5003bc87 0x55fab7793cda
Substituting fallback discounts for order 0: D1=0.5 D2=1 D3+=1.5
Substituting fal

In [7]:
pip install https://github.com/kpu/kenlm/archive/master.zip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 550 kB 19.5 MB/s 
  Created wheel for kenlm: filename=kenlm-0.0.0-cp37-cp37m-linux_x86_64.whl size=2372045 sha256=ad3e9c5c0188b12967523d5962e7cd0d8cf9a570f8b866ab256c6e069bc531cd
  Stored in directory: /tmp/pip-ephem-wheel-cache-tiyyc2w_/wheels/3d/aa/02/7b4a2eab5d7a2a9391bd9680dbad6270808a147bc3b7047e4e
Successfully built kenlm


In [1]:
import kenlm
model = kenlm.Model("/content/kenlm/build/text.arpa")
print(model.score('I is a student', bos = True, eos = True))

-8.371892929077148


In [2]:
import kenlm
model = kenlm.Model("/content/kenlm/build/text.arpa")
print(model.score('hello world everyone', bos = True, eos = True))

-1.826011300086975


In [3]:
import os
import kenlm

LM = '/content/kenlm/build/text.arpa'
model = kenlm.LanguageModel(LM)
print('{0}-gram model'.format(model.order))

sentence = 'hello world everyone'
print(sentence)
print(model.score(sentence))

# Check that total full score = direct score
def score(s):
    return sum(prob for prob, _, _ in model.full_scores(s))

assert (abs(score(sentence) - model.score(sentence)) < 1e-3)

# Show scores and n-gram matches
words = ['<s>'] + sentence.split() + ['</s>']
for i, (prob, length, oov) in enumerate(model.full_scores(sentence)):
    print('{0} {1}: {2}'.format(prob, length, ' '.join(words[i+2-length:i+2])))
    if oov:
        print('\t"{0}" is an OOV'.format(words[i+1]))

# Find out-of-vocabulary words
for w in words:
    if not w in model:
        print('"{0}" is an OOV'.format(w))

#Stateful query
state = kenlm.State()
state2 = kenlm.State()
#Use <s> as context.  If you don't want <s>, use model.NullContextWrite(state).
model.BeginSentenceWrite(state)
accum = 0.0
accum += model.BaseScore(state, "a", state2)
accum += model.BaseScore(state2, "sentence", state)
#score defaults to bos = True and eos = True.  Here we'll check without the end
#of sentence marker.  
assert (abs(accum - model.score("a sentence", eos = False)) < 1e-3)
accum += model.BaseScore(state, "</s>", state2)
assert (abs(accum - model.score("a sentence")) < 1e-3)

2-gram model
hello world everyone
-1.826011300086975
-1.007190465927124 2: <s> hello
-0.28817203640937805 2: hello world
-0.28817203640937805 2: world everyone
-0.24247679114341736 2: everyone </s>


In [4]:
import os
import kenlm

LM = '/content/kenlm/build/text.arpa'
model = kenlm.LanguageModel(LM)
print('{0}-gram model'.format(model.order))

sentence = 'I is a student'
print(sentence)
print(model.score(sentence))

# Check that total full score = direct score
def score(s):
    return sum(prob for prob, _, _ in model.full_scores(s))

assert (abs(score(sentence) - model.score(sentence)) < 1e-3)

# Show scores and n-gram matches
words = ['<s>'] + sentence.split() + ['</s>']
for i, (prob, length, oov) in enumerate(model.full_scores(sentence)):
    print('{0} {1}: {2}'.format(prob, length, ' '.join(words[i+2-length:i+2])))
    if oov:
        print('\t"{0}" is an OOV'.format(words[i+1]))

# Find out-of-vocabulary words
for w in words:
    if not w in model:
        print('"{0}" is an OOV'.format(w))

#Stateful query
state = kenlm.State()
state2 = kenlm.State()
#Use <s> as context.  If you don't want <s>, use model.NullContextWrite(state).
model.BeginSentenceWrite(state)
accum = 0.0
accum += model.BaseScore(state, "a", state2)
accum += model.BaseScore(state2, "sentence", state)
#score defaults to bos = True and eos = True.  Here we'll check without the end
#of sentence marker.  
assert (abs(accum - model.score("a sentence", eos = False)) < 1e-3)
accum += model.BaseScore(state, "</s>", state2)
assert (abs(accum - model.score("a sentence")) < 1e-3)

2-gram model
I is a student
-8.371892929077148
-2.1033761501312256 1: I
	"I" is an OOV
-1.802346110343933 1: is
	"is" is an OOV
-1.5221662521362305 1: a
-2.1033761501312256 1: student
	"student" is an OOV
-0.8406283855438232 1: </s>
"I" is an OOV
"is" is an OOV
"student" is an OOV
